# Principal Component Analysis
### A Primer on the Math and Application of PCA for Neural Interface Signal Postprocessing

Katie Heath and Keita Yokoyama

Oct. 22, 2019 - BMME 890-014

Joint Dpt. of Biomedical Engineering, UNC-NCSU

# What is PCA?



**More on why feature scaling matters:**

scikit-learn.org/stable/auto_examples/preprocessing/plot_scaling_importance.html#sphx-glr-auto-examples-preprocessing-plot-scaling-importance-py

## Orthogonality

## Relation to Unsupervised Learning and Classification

## PCA as a ML Pre-Processing Step

## PCA's Utility for Complex Signals

# Study Design
* Eight volunteers (5 men, 3 women; mean age 58 ± 12 yrs.)

* "definite, probable, or probable with laboratory support ALS diagnosis (mean ALSFRS-R scores: 32 ± 8)"

* EEG
  * active Ag/AgCl electrodes
  * ref: right earlobe
  * gnd: left mastoid
  * 256 Hz acquisition + bandpass 0.1-30 Hz

## Protocol

* Prompt participantss with short words;
* Present 6 x 6 matrix of characters, and;
* Ask to focus attention on 1 character in the word at a time.

Every 250 milliseconds, a different row or column of this matrix was successively and randomly made brighter.

Two out of the 12 rows or columns (read: 1 row, and 1 column) contained the desired character.

### Protocol: Stimulus Repetition

The 12 stimulations constituted a single Stimulus Repetition...

<div>
<img src="protocol-1.png" width="800" alt="infographic contextualizing stimulus with respect to Stimulus Repetition">
</div>

### Protocol: Overall Study Design

...and 10 Stimulus Repetitions were administered for one of 35 Trials (letters).

<div>
<img src="protocol-2.png" width="800" alt="infographic for higher-order protocol structure">
</div>

# Partial Replication of *Riccio et al.*
## Load Dataset

In [2]:
from scipy.io import loadmat;   import numpy as np

for n in range(8):        # for each of 8 subjects...
    content=loadmat( '../Datasets/BMI_ALS/A0' + str(n+1) + '.mat' )['data'][0,0]
    
    trialNum=content[4]-1 # index for new trial, corrected from Matlab notation
    
    # NEED TO SEPARATE BELOW INTO INDIVIDUAL TRIALS/REPETITIONS/STIMULATIONS
    # (ignore "runs" and ignore the context of words, for this analysis)
    EEG_data=content[1]   # EEG waveform (sample x channel)
    stimType=content[2]   # stimulation type (0=none, 1=nontarget, 2=target)
    stimClas=content[3]   # stimulation class (1-12 for each column/row)

## Standard Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

# given some input "x"
#x = StandardScaler().fit_transform(x) # Standardizing the features

## Examine Raw Data
AKA: why looking at raw EEG data without context is pointless

## Reorganize Dataset by Trial

## Apply PCA

In [ ]:
from sklearn.decomposition import PCA

pca1=PCA(n_components=2)
x_pca1=pca1.fit_transform(x)

pca2=PCA(0.95)
x_pca2=pca2.fit_transform(x)
